In [29]:
from sklearn.datasets import load_iris

In [2]:
import pandas as pd
import numpy as np
import operator
import matplotlib.pyplot as plt

In [3]:
iris=load_iris()

In [4]:
iris.feature_names

['sepal length (cm)',
 'sepal width (cm)',
 'petal length (cm)',
 'petal width (cm)']

In [5]:
iris.target_names

array(['setosa', 'versicolor', 'virginica'], dtype='<U10')

In [6]:
data = pd.read_csv('iris.data', header=None, names=['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'class'])


In [7]:
data.head()

,sepal_length,sepal_width,petal_length,petal_width,class
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [8]:
data.isnull().sum()

sepal_length    0
sepal_width     0
petal_length    0
petal_width     0
class           0
dtype: int64

In [9]:
from sklearn.model_selection import train_test_split
data_features = data.drop("class", axis=1)
# split dataframe and income
X_train, X_test, y_train, y_test = train_test_split(data_features, data['class'], random_state=0)

In [10]:
def euclideanDistance(data_1, data_2, data_len):
    dist = 0
    for i in range(data_len):
        dist = dist + np.square(data_1[i] - data_2[i])
    return np.sqrt(dist)

In [11]:
def knn1(x_train, y_train, testInstance, k): 
    distances = []
    mapping = []
    length = testInstance.shape[1]
    for x in range(len(x_train)):
        dist_up = euclideanDistance(testInstance, x_train.iloc[x], length)
        val = dist_up[0]

        if(len(distances)==k):
            ind = 0
            for item in distances:
                if item > val:
                    distances.insert(ind, val)
                    distances.pop(-1)
                    mapping.insert(ind,x)
                    mapping.pop(-1)
                    break
                else:
                    ind+=1
        elif(len(distances)==0):
            distances.append(val)
            mapping.append(x)
        else:
            ind = 0
            flag=1
            for item in distances:
                if item > val:
                    distances.insert(ind, val)
                    mapping.insert(ind, x)
                    flag=0
                    break
                else:
                    ind+=1
            if flag==1:
                distances.append(val)
                mapping.append(x)
    sort_distances = []
    i = 0

    while(i<len(mapping)):
        sort_distances.append((mapping[i], distances[i]))
        i+=1
    neighbors = []
    # Extracting nearest k neighbors
    for x in range(k):
        neighbors.append(sort_distances[x][0])
    # Initializing counts for 'class' labels counts as 0
    counts = {"Iris-setosa" : 0, "Iris-versicolor" : 0, "Iris-virginica" : 0}
    # Computing the most frequent class
    for x in range(len(neighbors)):

        response = y_train.iloc[neighbors[x]]
        if response in counts:
            counts[response] += 1
        else:
            counts[response] = 1
    
    sort_counts = sorted(counts.items(), key=operator.itemgetter(1), reverse=True)
    return(sort_counts[0][0])

In [12]:
import numpy as np
import tqdm
row_list = []
for index, rows in X_train.iterrows():
    my_list =[rows.sepal_length, rows.sepal_width, rows.petal_length, rows.petal_width]       
    row_list.append([my_list])

k_n = [1, 3, 5, 7]
dist_method = 'euclidean'

obs_k = {}
development_set_obs_k = {}
for k in k_n:
    development_set_obs = []
    for i in tqdm.tqdm(range(len(row_list))):
        development_set_obs.append(knn1(X_train, y_train, pd.DataFrame(row_list[i]), k))
    development_set_obs_k[k] = development_set_obs
obs_k[dist_method] = development_set_obs_k

100%|██████████| 112/112 [00:09<00:00, 11.85it/s]


In [13]:
accuracy = {}
for key in obs_k.keys():
    accuracy[key] = {}
    for k_value in obs_k[key].keys():
        
        count = 0
        for i,j in zip(y_train, obs_k[key][k_value]):
            if i == j:
                count = count + 1
            else:
                pass
        accuracy[key][k_value] = count/(len(y_train))
print(accuracy)

{'euclidean': {1: 1.0, 3: 0.9642857142857143, 5: 0.9732142857142857, 7: 0.9732142857142857}}


In [14]:
# Solving with heap now 

In [15]:
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score
import heapq

In [16]:
iris=data.copy()

In [17]:
iris.head()

,sepal_length,sepal_width,petal_length,petal_width,class
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [18]:
x=iris.drop('class',axis=1)
y=iris['class']

In [19]:
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.33) 
# split dataset into train and test

In [20]:
# will give us predicted value from the heap, heap is already sorted
def pred(heap):
    cl=[a[1] for a in heap]
    return max(cl)

In [21]:
def knn(pt,x_train,y_train,k=3):
    pt_arr=np.tile(np.array(pt),(len(x_train),1))
    # will give us array of points and same shape as x_train
    eucl_dist=np.sum(np.square(pt_arr-x_train),axis=1)
    # get euclidean distance between point and dataset
    mh=[]
    # empty heap to store distance values
    for i in range (len(x_train)):
        heapq.heappush(mh,(eucl_dist.iloc[i],y_train.iloc[i]))
    topk=[mh[i] for i in range(k)]
    # heap list is sorted and we need top k elements
    return pred(topk)

In [23]:
y_pred=[]

In [24]:
for i in range(len(x_test)):
    y_pred.append(knn(x_test.iloc[i],x_train,y_train))

In [33]:
from sklearn.metrics import classification_report

In [34]:
print(classification_report(y_test,y_pred))

                 precision    recall  f1-score   support

    Iris-setosa       1.00      1.00      1.00        18
Iris-versicolor       1.00      0.88      0.94        17
 Iris-virginica       0.88      1.00      0.94        15

       accuracy                           0.96        50
      macro avg       0.96      0.96      0.96        50
   weighted avg       0.96      0.96      0.96        50



In [31]:
# if heap is used for searching, then the complexity becomes O(nlogn) for heap.push()
# where n is number of data points in the training set

In [32]:
# precision comes out to be 96% 
# heap complexity is O(N^2) as we use insert and heap.pop()

In [35]:
# collaborated with Manu Singhal

In [36]:
# references 
# https://towardsdatascience.com/how-to-build-knn-from-scratch-in-python-5e22b8920bd2